In [55]:
%pip install --upgrade --quiet docx2txt langchain-community
%pip install -qU langchain-text-splitters langchain-chroma 
%pip install langchain langchainhub

You should consider upgrading via the '/Users/righteous/.pyenv/versions/address-llm-application/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/righteous/.pyenv/versions/address-llm-application/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/righteous/.pyenv/versions/address-llm-application/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)
loader = Docx2txtLoader('./tax.docx')

# Docx2txtLoader를 사용하여 도큐먼트를 txt로 지정한다. 이때 text_splitter를 지정하면 chunk 사이즈 단위로 document를 나눌 수 있다.
document_list = loader.load_and_split(text_splitter=text_splitter)

In [30]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

# text-embedding-3-large 모델을 사용하여 emdedding 생성
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [31]:
from langchain_chroma import Chroma

# vector 데이터베이스에 embedding한 데이터를 저장한다.
database = Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory='./chroma')


In [34]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query=query, k=3)
retrieved_docs

[Document(id='36948d8d-6a9d-491f-9bea-707df54f43d0', metadata={'source': './tax.docx'}, page_content='[전문개정 2009. 12. 31.]\n\n\n\n제10조(납세지의 변경신고) 거주자나 비거주자는 제6조부터 제9조까지의 규정에 따른 납세지가 변경된 경우 변경된 날부터 15일 이내에 대통령령으로 정하는 바에 따라 그 변경 후의 납세지 관할 세무서장에게 신고하여야 한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n\n[전문개정 2009. 12. 31.]\n\n\n\n제2장 거주자의 종합소득 및 퇴직소득에 대한 납세의무 <개정 2009. 12. 31.>\n\n\n\n제1절 비과세 <개정 2009. 12. 31.>\n\n\n\n제12조(비과세소득) 다음 각 호의 소득에 대해서는 소득세를 과세하지 아니한다. <개정 2010. 12. 27., 2011. 7. 25., 2011. 9. 15., 2012. 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31., 2025. 10. 1.>\n\n1. 「공익신탁법」에 따른 공익신탁의 이익\n\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원

In [25]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [26]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다
- [Context]를 참고해서 사용자의 질문에 답변해주세요

[Context]
{retrieved_docs}

Question: {query}
"""

In [28]:
ai_message = llm.invoke(prompt)

In [29]:
ai_message.content

'연봉 5천만 원인 직장인의 소득세를 계산하기 위해선 몇 가지 절차를 따져봐야 합니다:\n\n1. **과세표준 계산**: 연봉 5천만 원에서 근로소득공제를 차감하여 과세표준을 계산합니다.\n2. **근로소득공제**: 근로소득공제는 소득 구간별로 다르며, 연봉 5천만 원의 경우 대략적으로 다음과 같은 공제가 적용될 수 있습니다(정확한 계산은 세법 변경 사항에 따라 다를 수 있으니, 최신 정보를 확인하시는 것이 좋습니다).\n3. **세율 적용**: 과세표준에 소득세율을 적용합니다. 한국의 소득세율은 progressive rate(누진세율) 구조로, 일반적으로 소득이 높을수록 높은 세율이 적용됩니다. 예를 들어, 일부 구간에서는 6%, 그 다음 구간에서는 15% 등으로 나뉩니다.\n4. **세액공제**: 최종 산출된 소득세에서 여러 가지 세액공제를 적용할 수 있습니다. 예를 들어, 인적 공제, 보험료 공제 등이 있습니다.\n\n다른 감면 사항이나 공제 항목(예: 인적 공제 및 보험료 공제)이 있을 수 있으므로 개인별로 최종 부담할 세금은 다를 수 있습니다. 따라서 정확한 소득세를 계산하기 위해서는 국세청 홈택스 등의 계산기를 이용하거나 세무 전문가의 도움을 받는 것이 좋습니다. \n\n또한, 소득 구간별 정확한 세율과 공제 항목은 매년 변경될 수 있으므로 최신 세법을 확인해야 합니다. '

In [62]:
from langchain_classic import hub 

prompt = hub.pull('rlm/rag-prompt')

In [63]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [70]:
from langchain_classic.chains import RetrievalQA


qa_chain = RetrievalQA.from_chain_type(
    llm,
    _chain_type_kwargs={"prompt": prompt},
    retrieved_docs=database.as_retriever()
)

ValidationError: 3 validation errors for RetrievalQA
retriever
  Field required [type=missing, input_value={'combine_documents_chain...ae0>, search_kwargs={})}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
_chain_type_kwargs
  Extra inputs are not permitted [type=extra_forbidden, input_value={'prompt': ChatPromptTemp...additional_kwargs={})])}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden
retrieved_docs
  Extra inputs are not permitted [type=extra_forbidden, input_value=VectorStoreRetriever(tags...1ae0>, search_kwargs={}), input_type=VectorStoreRetriever]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden